In [1]:
from keras.src.optimizers.schedules import ExponentialDecay
from keras.src.models.sequential import Sequential
from keras.src.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Input, BatchNormalization, Activation
from keras.src.optimizers import Adam
from keras.src.legacy.preprocessing.image import ImageDataGenerator
import tensorflow

from keras.src.optimizers import RMSprop,SGD,Adam
from keras.src.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from tensorflow.python.keras.engine import data_adapter

def _is_distributed_dataset(ds):
    return isinstance(ds, data_adapter.input_lib.DistributedDatasetSpec)

data_adapter._is_distributed_dataset = _is_distributed_dataset




In [2]:
img_height, img_width = 48, 48
batch_size = 64

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_data_gen = datagen.flow_from_directory(
    'Data/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode = 'grayscale',
    subset='training'
)
val_data_gen = datagen.flow_from_directory(
    'Data/test',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode = 'grayscale',
    subset='validation'
)



Found 22968 images belonging to 7 classes.
Found 1432 images belonging to 7 classes.


In [3]:
model = Sequential()
# model.add(Input(shape=(48,48,1)))
no_of_classes = 7
# model.add(
#     Conv2D(kernel_size=(3,3),
#     add(BatchNormalization()),
#     activation='relu',filters=32)
# )
# model.add(
#     Conv2D(kernel_size=(3,3),
#     model.add(BatchNormalization()),
#     activation='relu',filters=32)
# )
# model.add(
#     MaxPooling2D(pool_size=(2,2)),
#     add(BatchNormalization())
# )
# model.add(
#     Dropout(0.25)
# )
# model.add(
#     Conv2D(kernel_size=(3,3),
#     activation='relu',filters=32)
# )
# model.add(
#     MaxPooling2D(pool_size=(2,2)),
#     add(BatchNormalization())
# )

# model.add(
#     Conv2D(kernel_size=(3,3),
#     add(BatchNormalization()),
#     activation='relu',filters=32)
# )
# model.add(
#     MaxPooling2D(pool_size=(2,2))
# )
# model.add(
#     Dropout(0.25)
# )
# model.add(Flatten())
# model.add(
#     Dense(1024,activation='relu')
# )
# model.add(
#     Dropout(0.5)
# )
# model.add(
#     Dense(7,activation='softmax')
# )

model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#2nd CNN layer
model.add(Conv2D(128,(5,5),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#3rd CNN layer
model.add(Conv2D(512,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#4th CNN layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(no_of_classes, activation='softmax'))

model.summary()

# cv2.ocl.setUseOpenCL(False)

C:\Users\Student\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 48, 48, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 48, 48, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 48, 48, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 24, 24, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 24, 24, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 24, 24, 128)         │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 24, 24, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 24, 24, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 12, 12, 512)         │         590,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 12, 12, 512)         │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 12, 12, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 6, 6, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 6, 6, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 6, 6, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 6, 6, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 3, 3, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 4,478,727 (17.08 MB)

 Trainable params: 4,474,759 (17.07 MB)

 Non-trainable params: 3,968 (15.50 KB)

In [4]:

# checkpoint = ModelCheckpoint("/Documents/Emotion_Recognition//model.keras", monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# early_stopping = EarlyStopping(monitor='val_loss',
#                           min_delta=0,
#                           patience=3,
#                           verbose=1,
#                           restore_best_weights=True
#                           )

# reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
#                               factor=0.2,
#                               patience=3,
#                               verbose=1,
#                               min_delta=0.0001)

# callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

In [5]:
optimizer = Adam(learning_rate=0.01)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
emotion_model_info = model.fit(
    train_data_gen,
    steps_per_epoch=28709//128,
    validation_data=val_data_gen,
    epochs=25,
    validation_steps=7178//128,
)

Epoch 1/25


C:\Users\Student\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


224/224 ━━━━━━━━━━━━━━━━━━━━ 218s 943ms/step - accuracy: 0.2267 - loss: 2.1396 - val_accuracy: 0.2479 - val_loss: 1.8453
Epoch 2/25


C:\Users\Student\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


224/224 ━━━━━━━━━━━━━━━━━━━━ 132s 588ms/step - accuracy: 0.2961 - loss: 1.7797 - val_accuracy: 0.2598 - val_loss: 1.7984
Epoch 3/25
224/224 ━━━━━━━━━━━━━━━━━━━━ 219s 975ms/step - accuracy: 0.3687 - loss: 1.6128 - val_accuracy: 0.1983 - val_loss: 1.9962
Epoch 4/25
224/224 ━━━━━━━━━━━━━━━━━━━━ 130s 580ms/step - accuracy: 0.4403 - loss: 1.4580 - val_accuracy: 0.4239 - val_loss: 1.4923
Epoch 5/25
224/224 ━━━━━━━━━━━━━━━━━━━━ 217s 968ms/step - accuracy: 0.4636 - loss: 1.3915 - val_accuracy: 0.4874 - val_loss: 1.3563
Epoch 6/25
224/224 ━━━━━━━━━━━━━━━━━━━━ 134s 596ms/step - accuracy: 0.5001 - loss: 1.2962 - val_accuracy: 0.2277 - val_loss: 1.9001
Epoch 7/25
224/224 ━━━━━━━━━━━━━━━━━━━━ 216s 966ms/step - accuracy: 0.5134 - loss: 1.2752 - val_accuracy: 0.4811 - val_loss: 1.4063
Epoch 8/25
224/224 ━━━━━━━━━━━━━━━━━━━━ 132s 590ms/step - accuracy: 0.5369 - loss: 1.2313 - val_accuracy: 0.4141 - val_loss: 1.4135
Epoch 9/25
224/224 ━━━━━━━━━━━━━━━━━━━━ 206s 919ms/step - accuracy: 0.5586 - loss: 1.16

In [13]:
model_json = model.to_json()

with open('emotion_model.json','w') as json_file:
    json_file.write(model_json)


model.save_weights('emotion_model.weights.h5')

In [15]:
model.save('emotion_model.h5')